In [1]:
# import
import pandas as pd
import numpy as np
from simple.plotly import chartParallel, interactTable
from simple.funcs import vwap
from simple.jurik import JRSX
from simple.geneopt import GeneOpt
from multiprocessing import current_process
from simple.backtest import getLong, getShort, getProfit, npBacktestLimit

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
# declare chart linestyles
line_styles = {
    'Tick': dict(color='gray', opacity=0.25),
    'Center': dict(color='blue', opacity=0.5),
    'qA': dict(color='red', opacity=0.5, dash='dot'),
    'qB': dict(color='green', opacity=0.5, dash='dot'),
    'cA': dict(color='red', opacity=0.4),
    'cB': dict(color='green', opacity=0.4),
    'RSX': dict(color='orange', row=2),
    
    'Profit': dict(color='gray', width=3, opacity=0.4, secondary_y=True, shape='hv', connectgaps=True),
    'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
    'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1))
}

In [4]:
def model(Period: int = (1000, 10000), StdDev: float = (1, 4, 0.1), Threshold: int = (0, 20), Ratio: float = (0, 20, 0.1)):
    if Period == 0 or StdDev == 0: return 0, {}
    Tick = T.Price
    Center = vwap(T, Period)
    std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + std * StdDev
    qB = Center - std * StdDev
    RSX = JRSX(Tick, Period) - 50
    ask_correction = Threshold - RSX
    bid_correction = -Threshold - RSX
    cA = qA + ask_correction * Ratio
    cB = qB + bid_correction * Ratio

    trades = npBacktestLimit(T, cA, cB)
    Buy, Sell = getLong(trades), getShort(trades)
    P = getProfit(trades)
    Profit = {'x': P.Index, 'y': P.Profit.cumsum()}

    return P.Profit.sum(), {
        'Count': len(P),
        'AvgProfit': P.Profit.mean() if len(P) > 0 else 0,
        'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
     } if current_process().daemon else locals()

In [5]:
# Genetic optimizer
G = GeneOpt(model)
G.maximize(population_size=300, generations=5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'Period': 6713, 'StdDev': 3.5704355200724502, 'Threshold': 13, 'Ratio': 0.0}

In [6]:
# Genetic result browser
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Fitness')

interactTable(model, X, rows=2, **line_styles)

In [7]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'Period',
                              'values': array(…